In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models import OllamaLLM

template = """Question: {question}

Answer: Let's think step by step."""

prompt = ChatPromptTemplate.from_template(template)

model = OllamaLLM(model="llama3.1")

chain = prompt | model

chain.invoke({"question": "What is LangChain?"})

ImportError: cannot import name 'OllamaLLM' from 'langchain_community.chat_models' (/Users/sohanurrahman/Desktop/Projects/AICoverLetter/myenv/lib/python3.12/site-packages/langchain_community/chat_models/__init__.py)

In [4]:
from fpdf import FPDF

def generate_pdf(filled_cover_letter):
    # Generate PDF using FPDF
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Times", size=12)

    # Split the filled cover letter into lines and add to PDF
    for line in filled_cover_letter.split('\n'):
        pdf.cell(200, 10, txt=line, ln=True)

    pdf.output('data.pdf', 'F')

def extract_data(pdf_file):
    # Prepare the data for the template
    import fitz
    doc = fitz.open(pdf_file)
    text = ""
    for page in doc:
        text+=page.get_text()
    print(text)
    return text


job_desc = """
About the job
Meta is seeking AI Software Engineers to join our Research & Development teams. The ideal candidate will have industry experience working on Language related topics. The position will involve taking these skills and applying them to solve for some of the most crucial & exciting problems that exist on the web. We are hiring in multiple locations.

Software Engineer - Language (Technical Leadership) Responsibilities:

Apply relevant AI and machine learning techniques to build intelligent rich language systems that improve Meta's products and experiences
Assist in goal setting related to project impact, AI system design, and ML excellence
Develop custom/novel architectures, define use cases, and develop methodology & benchmarks to evaluate different approaches
Apply in-depth knowledge of how the machine learning system interacts with the other systems around it
Technically lead in a team environment across multiple scientific and engineering disciplines, making the architectural trade-offs required to rapidly deliver software solutions
Mentor other AI Engineers & improve the quality of AI work in the broader team
Drive the team's goals and technical direction to pursue opportunities that make your larger organization more efficient
Effectively communicate complex features and systems in detail
Write clean readable code, debug complex problems that span systems, prioritize ruthlessly and get things done with a high level of efficiency
Understand industry & company-wide trends to help assess & develop new technologies
Partner & collaborate with organization leaders to help improve the level of performance of the team & organization
Identify new opportunities for the larger organization & influence the appropriate people for staffing/prioritizing these new ideas

Minimum Qualifications:

Bachelor's degree in Computer Science, Computer Engineering, relevant technical field, or equivalent practical experience.
Specialized experience in one or more of the following machine learning/deep learning domains: Language: NLP, ASR, TTS, Conversational AI
Experience developing language algorithms or language infrastructure in C/C++ or Python

Preferred Qualifications:

Experience in deep learning and PyTorch
Experience with distributed systems or on-device algorithm development
Experience contributing to AI Publications
Proven track record of planning multi-year roadmap in which shorter-term projects ladder to the long term-vision
Experience in driving large cross-functional/industry-wide engineering efforts
Significant experience in mentoring/influencing senior engineers across organizations

About Meta:

Meta builds technologies that help people connect, find communities, and grow businesses. When Facebook launched in 2004, it changed the way people connect. Apps like Messenger, Instagram and WhatsApp further empowered billions around the world. Now, Meta is moving beyond 2D screens toward immersive experiences like augmented and virtual reality to help build the next evolution in social technology. People who choose to build their careers by building with us at Meta help shape a future that will take us beyond what digital connection makes possible today—beyond the constraints of screens, the limits of distance, and even the rules of physics.

Meta is proud to be an Equal Employment Opportunity and Affirmative Action employer. We do not discriminate based upon race, religion, color, national origin, sex (including pregnancy, childbirth, or related medical conditions), sexual orientation, gender, gender identity, gender expression, transgender status, sexual stereotypes, age, status as a protected veteran, status as an individual with a disability, or other applicable legally protected characteristics. We also consider qualified applicants with criminal histories, consistent with applicable federal, state and local law. Meta participates in the E-Verify program in certain locations, as required by law. Please note that Meta may leverage artificial intelligence and machine learning technologies in connection with applications for employment.


"""

# Setup and RAG implementation

In [39]:
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser
from langchain_community.vectorstores.lancedb import LanceDB
from langchain_experimental.text_splitter import SemanticChunker
from langchain.embeddings.huggingface import HuggingFaceEmbeddings  # for free embeddings

local_llm="llama3.2:3b"
# LLM
llm = ChatOllama(model=local_llm, format="json", temperature=0)

embedding = HuggingFaceEmbeddings(model_name = "BAAI/bge-small-en")  # or "BAAI/bge-large-en" for a larger model

# Resume text
pdf_file_path = 'Rahman.pdf'
resume_text = extract_data(pdf_file_path)

#job Description text
job_description = job_desc + resume_text


#  Initilize Text splitter
text_splitter = SemanticChunker(embedding)


docs_job = text_splitter.create_documents([job_description])
# docs_resume = text_splitter.create_documents([resume_text])
print(docs_job[0].page_content)



Sohanur Rahman
New York, New York | (929) 412 6398 | srahman06@manhattan.edu | Linkedin | GitHub
EDUCATION
Manhattan University
Riverdale, Bronx
Bachelor of Science in Computer Engineering | Engineering Honor Society | GPA: 3.52
Graduation Date: May 2025
WORK EXPERIENCE
Manhattan University
Remote
Research Assistant, Dr. Brent Horine
 - 
Jun 2024
Present
Worked on a Kaggle competition, processing a large dataset (150k images and CSV files) using PyTorch and CNN 
models (ResNet).
Preprocessed 3D stack images and metadata using Pandas for feature engineering, missing data handling, and 
normalization.
Applied SMOTE and scikit-learn for dataset balancing and imputation; developed and optimized a CNN pipeline with 
data augmentation, evaluated models, and prepared final submissions.
Headstarter AI Fellowship
Remote
Software Engineering Fellow | 6-week program
 - 
Jul 2024
Sep 2024
Developed projects using open-source LLAMA LLM, Node.js, and Next.js.
Participated in 2 hackathons, one of whi

# Setting up vectore Database

In [41]:
from langchain.vectorstores.chroma import Chroma # Importing Chroma vector store from Langchain
import shutil
import os

# Path to the directory to save Chroma database

def save_to_chroma(chunks, CHROMA_PATH):
  """
  Save the given list of Document objects to a Chroma database.
  Args:
  chunks (list[Document]): List of Document objects representing text chunks to save.
  Returns:
  None
  """

  # Clear out the existing database directory if it exists
  if os.path.exists(CHROMA_PATH):
    shutil.rmtree(CHROMA_PATH)

  # Create a new Chroma database from the documents using OpenAI embeddings
  db = Chroma.from_documents(
    chunks,
    embedding,
    persist_directory=CHROMA_PATH
  )

  # Persist the database to disk
  db.persist()
  print(f"Saved {len(chunks)} chunks to {CHROMA_PATH}.")

save_to_chroma(docs_job, "Chroma_jo")
# save_to_chroma(docs_resume, "Chroma_resume")


Saved 3 chunks to Chroma_jo.


In [46]:
from langchain_community.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate

PROMPT_TEMPLATE = """
Answer the question based only on the following context:
{context}
 - -
Answer the question based on the above context: {question}
"""

def query_rag(query_text, CHROMA_PATH):
    """
    Query a Retrieval-Augmented Generation (RAG) system using Chroma database and OpenAI.
    Args:
        - query_text (str): The text to query the RAG system with.
    Returns:
        - formatted_response (str): Formatted response including the generated text and sources.
        - response_text (str): The generated response text.
    """
    # Ensure embedding function is initialized
    embedding_function = embedding

    # Prepare the database
    db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

    # Retrieving the context from the DB using similarity search
    results = db.similarity_search_with_relevance_scores(query_text, k=3)



    # Check if there are any matching results or if the relevance score is too low
    if len(results) == 0 or results[0][1] < 0.5:  # Lower the threshold for testing
        return "Unable to find matching results.", None

    # Combine context from matching documents
    context_text = "\n\n - -\n\n".join([doc.page_content for doc, _score in results])

    # Create prompt template using context and query text
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, question=query_text)

    # Generate response text based on the prompt
    response_text = llm.predict(prompt)  # Get the raw text response (not JSON)


    # # Format and return response including generated text and sources
    formatted_response = f"{response_text}"
    return formatted_response

# # Example usage
# query_text = "Find valuable information from this job description such date, company name, projects, titles, motivation, specific responsibilities from the resume to goal of creating a cover letter"

# response_text = query_rag(query_text, "Chroma_resume")

# # Print final response
# print(response_text)




In [34]:
type(response_text)
import json

# response_text = json.loads(response_text)

# type(response_str)

for key, val in response_text.items():  
    print(f"Key: {key}, Value: {val}")


Key: date, Value: ['Sep 2024', 'Mar 2024', 'Jun 2023']
Key: company_name, Value: ['TaskSync', 'Independent Project', 'Association of Computing Machinery (ACM)', 'Google Developer Group On Campus', 'Manhattan University']
Key: projects, Value: ['Developed a task management system with a Python backend and REST API', 'Implemented secure user authentication using bcrypt for password hashing and pyotp for two-factor authentication (2FA) with Google Authenticator and QR codes']
Key: titles, Value: ['Software Engineer', 'Independent Project', 'President', 'Vice-President', 'Research Assistant']
Key: motivation, Value: ['Foster a vibrant tech community by hosting programming workshops, guest speaker events, and hands-on mini projects.', 'Empower students to explore cutting-edge fields and develop practical skills.']
Key: responsibilities, Value: ['Developed a task management system with a Python backend and REST API', 'Implemented secure user authentication using bcrypt for password hashing a

In [35]:
system = """
You are a professional cover letter writer. Your task is to customize and highlight relevant experience.

Sohanur Rahman   
srahman06@manhattan.edu  
(929) 412 6398  
{date}

Hiring Manager  
{company_name}

Dear Hiring Team,

I am excited to apply for the {job_title} position at {company_name}. With a deep passion for {industry/field}, I believe my background in {relevant_skills} positions me well for this role.

**Personal Motivation**:  
{user_motivation}.

Throughout my career, I have successfully demonstrated my ability to {specific_responsibilities} through my work in {relevant_experience}. One of my key achievements includes {quantifiable_result_or_achievement}, which I believe aligns closely with {company_name}'s goals.

I am particularly drawn to {company_name} because of {about_the_company} and its commitment to {company_values}. I admire how your organization {specific_company_initiative}, and I am eager to contribute to such initiatives.

Furthermore, I have been inspired by {referral_or_connection_optional}, who encouraged me to apply for this position. I am confident that my skills in {highlighted_skill} can significantly contribute to your team.

Thank you for considering my application. I look forward to the opportunity to discuss how my background, skills, and enthusiasms align with the needs of {company_name}.

Sincerely,  
Sohanur Rahman
"""


# from langchain_core.prompts import ChatPromptTemplate
# from langchain_ollama.llms import OllamaLLM

# template = """Question: {question}

# Answer: Let's think step by step."""

# prompt = ChatPromptTemplate.from_template(system)

# model = OllamaLLM(model="llama3.1")

# chain = prompt | model

# chain.invoke({"question": "Write a cover letter"})
# Generate the cover letter using the prompt
generated_cover_letter = system.format(response_text)

KeyError: 'date'

In [ ]:
"""
Billie Horton
(582) 341-9765
billie@horton.com

October 20, 2023

Jessie Jennings
Hiring Manager
{company_name}

Dear Hiring Manager,

I am writing to express my strong interest in the {Role_applying} at {company_name}. As a recent graduate with a degree in Computer Science and a passion for coding, I am excited about the opportunity to contribute to your innovative team and make a meaningful impact in the software development industry.

During my academic journey, I have gained a solid foundation in programming languages such as Java, C++, and Python. Through various projects and internships, I have developed a strong understanding of software development principles and best practices. I am particularly drawn to Re/codeRealm's commitment to cutting-edge technologies and its reputation for pushing boundaries in the industry. The prospect of working alongside talented professionals who share my passion for innovation is truly exhilarating.

What sets me apart as a candidate is my ability to think critically and solve complex problems. I thrive in fast-paced environments and enjoy collaborating with cross-functional teams to deliver high-quality software solutions. My experience working on a team project to develop a mobile application showcased my ability to adapt to changing requirements and deliver results under tight deadlines. I am confident that my strong analytical skills, attention to detail, and dedication to continuous learning make me a great fit for Re/codeRealm.

In addition to my technical skills, I possess excellent communication and interpersonal skills, which I believe are essential for effective collaboration and teamwork. I am a proactive and motivated individual who is always eager to learn and grow. I am confident that my passion for software engineering and my drive to excel will enable me to contribute to Re/codeRealm's success.

Thank you for considering my application. I am excited about the opportunity to further discuss how my skills and qualifications align with Re/codeRealm's vision. I have attached my resume for your review. I look forward to the possibility of joining your team and contributing to the development of innovative software solutions.

Sincerely,

Billie Horton"""

In [57]:
COVER_LETTER_TEMPLATE = """ You are a professional cover letter writer. Your task is to customize and highlight relevant experience.
Sohanur  
(929) 412-6398  
srahman06@manhattan.edu

October 20, 2023
 
Hiring Manager  
Google

Dear Hiring Manager,

I am excited to apply for the {Job_Title} position at {Company_Name}. {Personal_Motivation}

Throughout my career, I have successfully demonstrated my ability to {Specific_Responsibilities} through my work in {Relevant_Experience}. One of my key achievements includes {Quantifiable_Result}, which I believe aligns closely with {Company_Name}'s goals.

I am particularly drawn to {Company_Name} because {Custom_Paragraph}. 

Thank you for considering my application. I look forward to discussing how my background, skills, and enthusiasms align with the needs of {Company_Name}.

Sincerely,  
Sohanur
"""


In [50]:
query = query_rag("Find valuable information like job title, company name, company vison, goals & culture", "Chroma_jo")
print(query)

{ "job_title": "Software Engineer - Language (Technical Leadership)", "company_name": "Meta", "company_vision": "to build technologies that help people connect, find communities, and grow businesses", "company_goals": ["connect people", "find communities", "grow businesses"], "culture": "innovative and learning" }


In [52]:
type(query)
import json

def convert_to_json(query):
    query = json.loads(query)
    return query

query = convert_to_json(query)

In [53]:
for key, val in query.items():
    print(key, val)

job_title Software Engineer - Language (Technical Leadership)
company_name Meta
company_vision to build technologies that help people connect, find communities, and grow businesses
company_goals ['connect people', 'find communities', 'grow businesses']
culture innovative and learning


In [59]:
def generate_cover_letter(company_name, job_title, company_vision):
    # Gather required context for each paragraph
    personal_motivation = query_rag(f"Why am I excited about the {job_title} position at {company_name} for personal motivation? Based on the given informantion provide the answer", "Chroma_jo")
    specific_responsibilities = query_rag(f"What responsibilities have I handled that relate to {job_title}? Based on the given informantion provide the answer", "Chroma_jo")
    relevant_experience = query_rag(f"What relevant experiences do I have for {job_title} and how it match {company_vision}?", "Chroma_jo")
    quantifiable_result = query_rag(f"What quantifiable results have I achieved in my career?", "Chroma_jo")
    custom_paragraph = query_rag(f"Why am I particularly drawn to {company_name}?", "Chroma_jo")

    # Fill the template
    cover_letter = COVER_LETTER_TEMPLATE.format(
        Job_Title=job_title,
        Company_Name=company_name,
        Personal_Motivation=personal_motivation,
        Specific_Responsibilities=specific_responsibilities,
        Relevant_Experience=relevant_experience,
        Quantifiable_Result=quantifiable_result,
        Custom_Paragraph=custom_paragraph
    )

    return cover_letter
job_title = query["job_title"]
company_name = query["company_name"]
company_vision = query["company_vision"]
# Example usage
cover_letter = generate_cover_letter(company_name, job_title, company_vision)
print(cover_letter)


 You are a professional cover letter writer. Your task is to customize and highlight relevant experience.
Sohanur  
(929) 412-6398  
srahman06@manhattan.edu

October 20, 2023
 
Hiring Manager  
Google

Dear Hiring Manager,

I am excited to apply for the Software Engineer - Language (Technical Leadership) position at Meta. {"The candidate is not explicitly stating their reasons for being motivated by this job. However, based on the provided information, it can be inferred that the candidate's excitement about the Software Engineer - Language (Technical Leadership) position at Meta stems from their passion for language-related topics and their desire to apply their skills in a cutting-edge environment."  : "The candidate has experience working on language algorithms or language infrastructure in C/C++ or Python. They have also developed projects using open-source LLAMA LLM, Node.js, and Next.js, which suggests that they are interested in natural language processing (NLP) and machine lear

# New Approch

In [3]:
import os
from langchain.prompts import PromptTemplate
from langchain_ollama import ChatOllama

def generate_cover_letter_open_source(job_description, resume):
    # print(f"selected_model: {selected_model}, "
    #       f"{AVAILABLE_MODELS_GGUF[selected_model]['model_file']}, {AVAILABLE_MODELS_GGUF[selected_model]['model_type']}")
    # print(f"context_length: {context_length}")

    prompt = (f"Do the following steps: "
              f"1. Read the following job description,"
              f"2. Read the following resume, "
              f"3. Write a formal cover letter to the hiring manager for the job description based on the given resume, "
              # f"4. The cover letter MUST BE within {output_size_range[0]} to {output_size_range[1]} words. "
              # f"4. The cover letter MUST BE within 100 words. "
              f"4. Return ONLY the cover letter ONCE, nothing else. "
              f"Job Description: '{job_description}'. Resume: '{resume}'")

    # prompt = "What is an LLM"

    llm = ChatOllama(
    model="llama3.2:3b",
    temperature=0,
    # other params...
)

    llm_response = llm.invoke(prompt)

    return llm_response


In [4]:
job_description = """

Job description
This job is with JPMorganChase, an inclusive employer and a member of myGwork – the largest global platform for the LGBTQ+ business community. Please do not contact the recruiter directly.

We have an exciting and rewarding opportunity for you to take your software engineering career to the next level.

As a Software Engineer III at JPMorgan Chase within the Consumer and Community Banking, within Architecture and Engineering, you serve as a seasoned member of an agile team to design and deliver trusted market-leading technology products in a secure, stable, and scalable way. You are responsible for carrying out critical technology solutions across multiple technical areas within various business functions in support of the firm’s business objectives.

Job Responsibilities
• Executes software solutions, design, development, and technical troubleshooting with ability to think beyond routine or conventional approaches to build solutions or break down technical problems
• Creates secure and high-quality production code and maintains algorithms that run synchronously with appropriate systems
• Produces architecture and design artifacts for complex applications while being accountable for ensuring design constraints are met by software code development
• Gathers, analyzes, synthesizes, and develops visualizations and reporting from large, diverse data sets in service of continuous improvement of software applications and systems
• Proactively identifies hidden problems and patterns in data and uses these insights to drive improvements to coding hygiene and system architecture
• Contributes to software engineering communities of practice and events that explore new and emerging technologies
• Adds to team culture of diversity, equity, inclusion, and respect

Required Qualifications, Capabilities, And Skills
• Formal training or certification on software engineering concepts and 3+ years applied experience
• Proficiency in one or more modern programming languages such as; Java, Selenium, GIT, Jenkins, WebDriver, TestNG, IOS and Android
• Experience in building Restful webservices using rest client.
• Implemented Java or Python for developing the automation framework for API testing
• Testing new fixes using GIT and Jenkins after a build is kicked off part of Continuous Integration Testing
• Implemented Appium framework in order to test IOS and Android Mobile application.
• Implemented Page Object Model Automation framework with Java, Selenium WebDriver and TestNG
• Attending Sprint plan and Sprint Review meetings, daily standup, Demo and Retrospective meeting
• Creating framework, fixing framework, fixing and writing scripts using Java and Selenium WebDriver
• Performing POC on request and integrate the same in DevOps pipeline
• Experience with unit testing and automated testing frameworks like jest, jasmine and puppeteer

Preferred Qualifications, Capabilities, And Skills
• Familiarity with modern front-end technologies
• Exposure to cloud technologies
• Understanding of software skills such as business analysis, development, maintenance and software improvement

"""

resume = """

Sohanur Rahman
New York, New York | (929) 412 6398 | srahman06@manhattan.edu | Linkedin | GitHub
EDUCATION
Manhattan University Riverdale, Bronx
Bachelor of Science in Computer Engineering | Engineering Honor Society | GP A: 3.52 Graduation Date: May 2025
WORK EXPERIENCE
Manhattan University Remote
Research Assistant, Dr. Brent Horine
Jun 2024 Present
-
Worked on a Kaggle competition, processing a large dataset (150k images and CSV files) using PyTorch and CNN
models (ResNet).
Preprocessed 3D stack images and metadata using Pandas for feature engineering, missing data handling, and
normalization.
Applied SMOTE and scikit-learn for dataset balancing and imputation; developed and optimized a CNN pipeline with
data augmentation, evaluated models, and prepared final submissions.
Headstarter AI Fellowship Remote
Software Engineering Fellow | 6-week program
Jul 2024 Sep 2024
-
Developed projects using open-source LLAMA LLM, Node.js, and Next.js.
Participated in 2 hackathons, one of which was a finalist, solving real-world startup challenges.
Led backend development using Node.js, AWS DynamoDB, and Lambda to build an internal feedback tool that
summarized key points from customer call transcripts in real-time.
Teads Inc. IT Support Engineer intern
New York, New York
Jun 2023 Dec 2023
-
Resolved 200+ hardware and software issues during a 6-month internship, achieving a 95% satisfaction rating
through effective use of diagnostic tools and excellent troubleshooting skills.
Upgraded a large-scale network to boost system reliability and reduce the volume of help-desk tickets.
Automated updates and security checks using Python and PowerShell, and implemented MDM tools with RSA
certificates.
PROJECT EXPERIENCE
TaskSync | GitHub Full-Stack Engineer
Riverdale, New York
Sep 2024 Present
-
Developed a task management system with a Python backend and REST API, implementing role-based access control
and Firebase integration for authentication and data management.
Designed and implemented a user-friendly React Native interface, ensuring real-time data synchronization with
Firebase.
Conducted API testing using Postman and managed version control with GitHub, coordinating smooth integration
between the frontend and backend.
Secure Web Authentication System | GitHub Independent Project
New York
Mar 2024 May 2024
-
Implemented secure user authentication using bcrypt for password hashing and pyotp for two-factor authentication
(2FA) with Google Authenticator and QR codes.
Utilized the cryptography library for data encryption and ensured efficient local data storage with SQLite.
Create YouTube Shorts Using Reddit | GitHub Independent Project
New York
Jan 2023 Mar 2023-
Developed a Python-based system integrating Reddit API, MoviePy, and YouTube API to automatically generate and
upload YouTube Shorts using Reddit images, comments, and background music.
Automated the end-to-end process from content extraction to video creation and uploading, enhancing content
engagement by curating trending Reddit content for social media.
LEADERSHIP EXPERIENCE
Association of Computing Machinery (ACM) President
Riverdale, New York
Sep 2024 Present
-
Lead the ACM chapter to foster a vibrant tech community by hosting programming workshops, guest speaker events,
and hands-on mini projects.
Introduce emerging technologies to students, empowering them to explore cutting-edge fields and develop practical
skills.
Collaborate with non-technical clubs to demonstrate the value of technology and programming across diverse
disciplines.
Google Developer Group On Campus, Manhattan Univer Vice-President
Riverdale
Sep 2024 Present
-
Provided support and guidance to organize and run AI workshops, helping students gain hands-on experience with
cutting-edge technologies.
Managed and built a strong tech community on campus by collaborating with various student organizations and tech
leaders, fostering a culture of innovation and learning.
Offered mentorship and direction to tech leads and e-board members, ensuring successful collaboration and effective
leadership across multiple projects.
SKILLS & ACTIVITY
AWS (Lambda, DynamoDB), Google Cloud, PyTorch, TensorFlow, Langchain, Docker, SQLite, PostgreSQL,
Tools
:
Supabase, Firebase, Postman, Pandas, Matplotlib
Language :
C/C++, Python. Javascript, Java, React/ReactNative, NextJS, NodeJS
"""

generate_cover_letter_open_source(job_description, resume)

AIMessage(content="Dear Hiring Manager,\n\nI am excited to apply for the Software Engineer III position at JPMorgan Chase within the Consumer and Community Banking, Architecture and Engineering department. With a strong foundation in software engineering concepts and 3+ years of applied experience, I am confident that my skills and passion for innovation make me an ideal candidate for this role.\n\nAs a seasoned member of agile teams, I have a proven track record of designing and delivering trusted market-leading technology products in a secure, stable, and scalable way. My experience in building Restful web services using REST client, implementing Java or Python for developing automation frameworks for API testing, and attending sprint planning and review meetings has equipped me with the skills to contribute to software engineering communities of practice and events that explore new and emerging technologies.\n\nI am particularly drawn to this role because of JPMorgan Chase's commitm